# Answering Business Questions using SQL

For this project we will work with a database called [Chinook](https://app.dataquest.io/m/189/intermediate-joins-in-sql/1/working-with-larger-databases) which is a bout a fictional digital music shop - kind of like a mini-iTunes store.
the Chinook database contains information about artists,songs, also information on the shop's employees, customers and the customers purchases, it is made of 11 tables
We will at first explore the tables and then we will respond some questions

<img src=https://s3.amazonaws.com/dq-content/191/chinook-schema.svg>

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

## Exlporing dataset

First let´s explore the tables we have in our dataset

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql
SELECT
    *
  FROM employee,customer
 LIMIT 5; 

Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email,customer_id,first_name_1,last_name_1,company,address_1,city_1,state_1,country_1,postal_code_1,phone_1,fax_1,email_1,support_rep_id
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


In [4]:
%%sql
SELECT *
  FROM invoice,invoice_line
LIMIT 5; 

Done.


invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total,invoice_line_id,invoice_id_1,track_id,unit_price,quantity
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84,1,1,1158,0.99,1
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84,2,1,1159,0.99,1
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84,3,1,1160,0.99,1
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84,4,1,1161,0.99,1
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84,5,1,1162,0.99,1


## New albums to purchase

The Chinook record store has just signed a deal with a new record label and we need to select the first three albums that will be added to the store from 4 albums.
The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA. We will find out which genres sell the most tracks in USA

Artist Name  | Genre
------------ | -----
Regal       |  Hip-Hop
Red Tone	 | Punk
Meteor and the Girls| Pop
Slim Jim Bites|	Blues

In [5]:
%%sql
WITH
    total_us AS
            (
             SELECT il.* FROM invoice_line il
             INNER JOIN invoice i on il.invoice_id = i.invoice_id
             INNER JOIN customer c on i.customer_id = c.customer_id
             WHERE c.country = "USA"
            )
SELECT 
     g.name AS genre,
     COUNT(tu.invoice_line_id) AS tracks_sold,
     ROUND(CAST(count(tu.invoice_line_id) AS FLOAT) / (
        SELECT COUNT(*) FROM total_us
    ),2) AS percentage_sold
    
FROM total_us AS  tu
INNER JOIN track t on t.track_id = tu.track_id
INNER JOIN genre g on g.genre_id = t.genre_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;

Done.


genre,tracks_sold,percentage_sold
Rock,561,0.53
Alternative & Punk,130,0.12
Metal,124,0.12
R&B/Soul,53,0.05
Blues,36,0.03
Alternative,35,0.03
Latin,22,0.02
Pop,22,0.02
Hip Hop/Rap,20,0.02
Jazz,14,0.01


Based on the previous query we can say that the albums to purchase will be:

   *Red Tone(Punk)
   *Slim Jim Bites(Blues)
   *Meteor and the Girls(Pop)
   
Combine these three albums just represent 17% of total US market compared with rock which represent the 53% of the total USA market

##  Employee Sales Performance

As we can say before we have a table which show data of employees, now we will find out the total amount of sales for each employee and the hire date 

In [6]:
%%sql
SELECT 
    e.first_name || " " || e.last_name AS "Employee name",
    e.hire_date AS "Hire date",
    ROUND(sum(i.total),2) AS "Total dollar amount of sales"
FROM employee AS e
INNER JOIN customer AS c ON c.support_rep_id=employee_id
INNER JOIN invoice AS i ON i.customer_id=c.customer_id
GROUP BY 1

Done.


Employee name,Hire date,Total dollar amount of sales
Jane Peacock,2017-04-01 00:00:00,1731.51
Margaret Park,2017-05-03 00:00:00,1584.0
Steve Johnson,2017-10-17 00:00:00,1393.92


Jane Peacock is the employee with most sales and also the oldest employee in the company as per hire date

## Total sales by country

Now we will get the total total value of sales, total number of customers, average value of sales per customerand average order value by country in order to explore markets with less customers but with a high average order. As there are countries with 1 customer only we will group them in a group called as other and put them in the bottom of the chart

In [7]:
%%sql
WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )

SELECT
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM
    (
    SELECT
        country,
        count(distinct customer_id) customers,
        ROUND(SUM(unit_price),2) total_sales,
        ROUND(SUM(unit_price) / count(distinct customer_id),2) customer_lifetime_value,
        ROUND(SUM(unit_price) / count(distinct invoice_id),2) average_order,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );

Done.


country,customers,total_sales,average_order,customer_lifetime_value
USA,13,1040.49,7.94,80.04
Canada,8,535.59,7.05,66.95
Brazil,5,427.68,7.01,85.54
France,5,389.07,7.78,77.81
Germany,4,334.62,8.16,83.66
Czech Republic,2,273.24,9.11,136.62
United Kingdom,3,245.52,8.77,81.84
Portugal,2,185.13,6.38,92.57
India,2,183.15,8.72,91.58
Other,15,1094.94,7.45,73.0


As per the previous results we can say that czech republic,United Kingdom and India despite they have few customers they have a high average order  and also a good level of total buy by customer.

## Sales based on albums vs tracks

The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

* purchase a whole album
* purchase a collection of one or more individual tracks.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

In order to answer the question, we're going to have to identify whether each invoice has all the tracks from an album. We can do this by getting the list of tracks from an invoice and comparing it to the list of tracks from an album. We can find the album to compare the purchase to by looking up the album that one of the purchased tracks belongs to. It doesn't matter which track we pick, since if it's an album purchase, that album will be the same for all tracks.

In [8]:
%%sql
WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;

Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


The results are contudent, customers do not buy the entire album, the 81% purchases of customers are based on purchase by tracks, only the 19% buy the complete album and not the songs.
I would recommend to the management buy tracks from record companies.